## Processing Corporate Actions as native LUSID transitions

In this notebook, we will show how you can book corporate action transactions into LUSID using LUSID's native corporate actions functionality. For the purposes of this demo, we will show:

* A dividend payment in cash
* A dividend reinvestment in stock

### Setup LUSID

In [1]:
import os
from datetime import datetime
import pytz

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import format_transactions_response

# Import data wrangling packages
import pandas as pd
import numpy as np
import json

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [2]:
# Define a function which formats responses from the corporate actions API


def corporate_action_request_details(ca):
    print(
        colours.bold
        + "CA Code and Type: "
        + colours.end
        + str(ca["code"])
        + str(ca["action_description"])
    )
    print(
        colours.bold
        + "Announcement Date : "
        + colours.end
        + str(ca["announcement_date"])
    )
    print(colours.bold + "Ex Date : " + colours.end + str(ca["ex_date"]))
    print(colours.bold + "Record Date : " + colours.end + str(ca["record_date"]))
    print(colours.bold + "Payment Date : " + colours.end + str(ca["payment_date"]))
    print(
        colours.bold
        + "input instrument : "
        + colours.end
        + str(ca["input_instrument_luid"])
    )
    print(
        colours.bold
        + "Units in : "
        + colours.end
        + str(ca["input_units_factor"])
        + colours.bold
        + " Cost in : "
        + colours.end
        + str(ca["input_cost_factor"])
    )
    print(
        colours.bold
        + "output instrument : "
        + colours.end
        + str(ca["output_instrument_luid"])
    )
    print(
        colours.bold
        + "output internal : "
        + colours.end
        + str(ca["output_instrument_internal"])
    )
    print(
        colours.bold
        + "Units out : "
        + colours.end
        + str(ca["output_units_factor"])
        + colours.bold
        + " Cost out : "
        + colours.end
        + str(ca["output_cost_factor"])
    )
    print("")


class colours:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    end = "\033[0m"
    bold = "\033[1m"
    UNDERLINE = "\033[4m"

### Load a sample portfolio

In this section we seed a new sample portfolio with 10 FTSE100 stocks and GBP cash. This portfolio will be used to demonstrate the corporate actions.

In [3]:
# Create a new scope

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

(For more information on scopes, please see the [scopes](https://support.finbourne.com/what-is-a-scope-in-lusid-and-how-is-it-used) documenation on the support page)

In [4]:
# Load a file of equity transactions

transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [5]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

print(f"Portfolio {portfolio_code} has been created with transactions.")

Portfolio EQUITY_UK has been created with transactions.


### Build a Transaction Portfolios API

We build a Transactions Portfolios API object so we can interact with the Transaction Portfolio methods. See the [API documentation](https://www.lusid.com/docs/api/#tag/Transaction-Portfolios) for a list of methods. 

In [6]:
# Create a transaction portfolios API

transactions_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [7]:
# View the holdings for today

response = transactions_portfolios_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(response, rename_properties=True)

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,385b-17ec-0d4e-0a/EQUITY_UK,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7XM08GZF,{},BHP,385b-17ec-0d4e-0a/EQUITY_UK,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_STGB38I6,{},Barclays,385b-17ec-0d4e-0a/EQUITY_UK,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_PVOJGULG,{},BP,385b-17ec-0d4e-0a/EQUITY_UK,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_Z57YKL4W,{},HSBC,385b-17ec-0d4e-0a/EQUITY_UK,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,LUID_RNHEK2PL,{},Morrisons,385b-17ec-0d4e-0a/EQUITY_UK,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_99M6G8U7,{},Tesco,385b-17ec-0d4e-0a/EQUITY_UK,P,200000.0,200000.0,400000.0,GBP,0.0,GBP
7,LUID_6JES517Q,{},Rightmove,385b-17ec-0d4e-0a/EQUITY_UK,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_7KLNIUU7,{},vodafone,385b-17ec-0d4e-0a/EQUITY_UK,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_NE84MHW9,{},Anglo American plc,385b-17ec-0d4e-0a/EQUITY_UK,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


### Create corporate actions souce

The corporate actions souce is a container for holding corporate actions. In this section, we:

* Create a new corporate actions source
* Assign corporate actions source to our portfolio

In [8]:
ca_source_code = "ca_demo"

# Create first corporate action source
try:

    source1_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code="ca_demo",
        display_name=ca_source_code,
        description="CASource 1 for python notebook demo module",
    )
    source1_result = api_factory.build(
        lusid.api.CorporateActionSourcesApi
    ).create_corporate_action_source(source1_request)
    CASource1 = source1_result.id
    print("Corporate Action Source Created : ")
    print("Disply Name: ", source1_result.display_name)
    print("Description: ", source1_result.description)
    print("Scope: ", CASource1.scope)
    print("Code: ", CASource1.code + "\n")

except:
    pass

Corporate Action Source Created : 
Disply Name:  ca_demo
Description:  CASource 1 for python notebook demo module
Scope:  385b-17ec-0d4e-0a
Code:  ca_demo



In [9]:
# Assign the corporate actions source to our portfolio

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at="2018-01-01T00:00:00+00:00",
    portfolio_details=lusid.models.CreatePortfolioDetails(
        corporate_action_source_id=lusid.ResourceId(scope=scope, code=ca_source_code)
    ),
)

print(
    f"Corporate actions source of {ca_source_code} assigned to portfolio {portfolio_code}"
)

Corporate actions source of ca_demo assigned to portfolio EQUITY_UK


### Load transitions from an external source

Transitions determine which Instrument is taking part in a corporate action, and what the effect of the corporate action is on holdings in that Instrument. 

For the transitions in our current example, we have:

* A bonus issue of 13,200 units (1 bonus share per 10 held)
* A cash dividend of £204,000 from BHP (£1.70 GBP per share held)

These transitions are posted with a payment date of 1 March 2020.

In [10]:
# Load transitions fron a CSV file

corpact_df = pd.read_csv("data/corpact_transitions.csv")
corpact_df

,code,action_description,description,announcement_date,ex_date,record_date,payment_date,client_id,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield
0,5943592342,dividend-cash,BHP,31/03/2020,31/03/2020,31/03/2020,31/03/2020,EQ_1235,BHP,1,1,CCY_GBP,1.7,0,1.7
1,5943592343,bonus-issue,Aviva,31/03/2020,31/03/2020,31/03/2020,31/03/2020,EQ_1234,Aviva,1,0,EQ_1234,1.1,0,0.0


In [11]:
# Add environment specific output LUIDs to the data set

out_instrument_luid = []

for index, item in corpact_df.iterrows():
    if item["output_instrument_internal"].startswith("CCY_"):
        out_instrument_luid.append("nan")
    else:
        out_instrument_luid.append(
            api_factory.build(lusid.api.InstrumentsApi)
            .get_instrument(
                identifier_type="ClientInternal",
                identifier=item["output_instrument_internal"],
            )
            .lusid_instrument_id
        )

corpact_df["output_instrument_luid"] = out_instrument_luid

In [12]:
# Add environment specific input LUIDs to the data set

corpact_df["input_instrument_luid"] = corpact_df["client_id"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi)
    .get_instrument(identifier_type="ClientInternal", identifier=x)
    .lusid_instrument_id
)

### Transalate transition into LUSID model for upload

In [13]:
import math

transitions = {}
actions = []

LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId"

for index, ca in corpact_df.iterrows():
    # Print the details contained in the CSV for each corporate action
    corporate_action_request_details(ca)

    # create transition components
    cat_in = models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={
            LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
        },
        units_factor=ca["input_units_factor"],
        cost_factor=ca["input_cost_factor"],
    )

    # Determine if the output is Cash or Stock, and create the appropriate transition component
    if str(ca["output_instrument_luid"]) == "nan":
        print("success")
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Currency": ca["output_instrument_internal"][4:]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
    else:
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )

    key = ca["code"]
    transitions.setdefault(key, [])
    transitions[key].append(ca["action_description"])
    transitions[key].append(cat_in)
    transitions[key].append(cat_out)

CA Code and Type: 5943592342dividend-cash
Announcement Date : 31/03/2020
Ex Date : 31/03/2020
Record Date : 31/03/2020
Payment Date : 31/03/2020
input instrument : LUID_7XM08GZF
Units in : 1 Cost in : 1
output instrument : nan
output internal : CCY_GBP
Units out : 1.7 Cost out : 0

success
CA Code and Type: 5943592343bonus-issue
Announcement Date : 31/03/2020
Ex Date : 31/03/2020
Record Date : 31/03/2020
Payment Date : 31/03/2020
input instrument : LUID_ATFGUBHS
Units in : 1 Cost in : 0
output instrument : LUID_ATFGUBHS
output internal : EQ_1234
Units out : 1.1 Cost out : 0



In [14]:
# Function to convert strings to date objects with timezones
def str_to_tzdate(dtstring):
    date_time_obj = datetime.strptime(dtstring, "%d/%m/%Y")
    timezone = pytz.utc
    timezone_date_time_obj = timezone.localize(date_time_obj)
    return timezone_date_time_obj


# Iterate through the transitions, turning them into LUSID Corporate Action Requests
for key, values in transitions.items():
    transition_code = key
    transition_type = values[0]
    transition_in = values[1]
    transitions_out = values[2:]

    for x in transitions_out:
        if isinstance(x, str):
            transitions_out.remove(x)

    temp_transition = models.CorporateActionTransition(
        input_transition=transition_in, output_transitions=transitions_out
    )

    # Extract the data for the corporate action from the LUID corporate actions dataframe
    data = corpact_df[corpact_df["code"] == transition_code]

    # Iterate through each row of data and create the Corporate Action Request for the appropriate type
    # of action.
    for row, d in data.iterrows():
        if transition_type == "dividend-cash":
            div_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d["code"]),
                announcement_date=str_to_tzdate(d["announcement_date"]),
                ex_date=str_to_tzdate(d["ex_date"]),
                record_date=str_to_tzdate(d["record_date"]),
                payment_date=str_to_tzdate(d["payment_date"]),
                transitions=[temp_transition],
            )
        if transition_type == "bonus-issue":
            split_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(d["code"]),
                announcement_date=str_to_tzdate(d["announcement_date"]),
                ex_date=str_to_tzdate(d["ex_date"]),
                record_date=str_to_tzdate(d["record_date"]),
                payment_date=str_to_tzdate(d["payment_date"]),
                transitions=[temp_transition],
            )

### Upsert the corporate actions into LUSID's movements engine

In [15]:
result = api_factory.build(
    lusid.api.CorporateActionSourcesApi
).batch_upsert_corporate_actions(
    scope=scope, code=ca_source_code, actions=[div_ca, split_ca]
)

### Holdings in Aviva and GBP cash on 31 January 2020 (before the Corporate Action)

* Portfolio has 132,000 units of Aviva
* Portfolio has £2,960,000 GBP cash

In [16]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-01-31",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,385b-17ec-0d4e-0a/EQUITY_UK,P,132000.0,132000.0,660000.0,GBP,0.0,ZZZ
10,CCY_GBP,{},CCY_GBP,385b-17ec-0d4e-0a/EQUITY_UK,B,2960000.0,2960000.0,2960000.0,GBP,0.0,ZZZ


### Holdings in Aviva and GBP cash on 30 April 2020 (after the Corporate Action)

* Portfolio has 145,200 units of Aviva (+13,200 units)
* Portfolio has £3,164,000 GBP cash (+ £204,000 cash)

In [17]:
response = transactions_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-04-30",
)

holdings_df = lusid_response_to_data_frame(response, rename_properties=True)
holdings_df[holdings_df["Name(default-Properties)"].isin(["Aviva", "CCY_GBP"])]

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,385b-17ec-0d4e-0a/EQUITY_UK,P,145200.0,145200.0,660000.0,GBP,0.0,ZZZ
10,CCY_GBP,{},CCY_GBP,385b-17ec-0d4e-0a/EQUITY_UK,B,3164000.0,3164000.0,3164000.0,GBP,0.0,ZZZ
